# 앙상블(ensemble) 모형

서로 다른 특성을 갖는 이질적인 모형(heterogeneous model)을 분류문제(classification)에 대해서 투표(voting)를 통해 예측모형을 개발하거나 평균을 내서 구축한 모형이 단일 모형을 사용하는 경우보다 훨씬 더 좋은 예측성능을 내고 있다.

최적 단일 모형을 개발하는 대신에 의사결정나무(decision tree) 모형을 다양하게 조합하여 예측력을 높이는 방법을 살펴보자.

# 기본 예측모형

분류(classification)나 회귀(regression) 모형의 경우 데이터의 전처리 작업이 완료되었다고 가정하고 나면 훈련/시험 데이터셋으로 구분하고 이를 다양한 예측모형을 적합시켜 기본 예측모형을 개발한다.

## 데이터 가져오기

캐글 [Mushroom Classification, Safe to eat or deadly poison?](https://www.kaggle.com/uciml/mushroom-classification) 웹사이트에서 버섯 분류 모형을 개발해보자. 이를 위해서 데이터 다운로드 받아 압축을 풀고 "data/mushrooms.csv" 저장한다. 그리고, 가장 별점을 많이 받은 [Comparing Various ML models(ROC curve comparison)](https://www.kaggle.com/nirajvermafcb/comparing-various-ml-models-roc-curve-comparison) 코드를 참조해 기계학습 예측모형에 필요한 전처리 작업 준비를 한다.

In [22]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

data = pd.read_csv("data/mushrooms.csv")
data.head(6)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


## 데이터 전처리

버섯종별로 식용 버섯과 독버섯으로 나눠져 있는데 모두 문자열(string)로 되어 있어 이를 일괄적으로 숫자로 변환시킨다.

In [23]:
from sklearn.preprocessing import LabelEncoder

labelencoder=LabelEncoder()

for col in data.columns:
    data[col] = labelencoder.fit_transform(data[col])
 
data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


## 훈련/시험 데이터 분리

다음으로 데이터프레임을 훈련 데이터와 시험데이터로 구분한다.
그리고, `X` 데이터프레임을 정규화시켜 특정 변수가 척도(scale) 때문에 예측에 영향을 과도하게 미치는 것을 방지시킨다.

In [24]:
X = data.iloc[:,1:23]  # all rows, all the features and no labels
y = data.iloc[:, 0]  # all rows, label only
X.head()
y.head()

# Scale the data to be between -1 and 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

## 단일 예측모형

분류문제(classification)에 대해서 가장 많이 알려진 의사결정나무, 로지스틱 회귀모형, 나이브 베이즈, K-이웃 분류모형, SVM을 각각 적합시켜본다.

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# 예측모형 초기화
clf_knn = KNeighborsClassifier(5)
clf_dt = DecisionTreeClassifier()
clf_lr = LogisticRegression()
clf_svm = SVC()
clf_nb = GaussianNB()

# 예측모형 적합
clf_knn.fit(X_train, y_train)
clf_dt.fit(X_train, y_train)
clf_lr.fit(X_train, y_train)
clf_svm.fit(X_train, y_train)
clf_nb.fit(X_train, y_train)

# 예측값
pred_knn = clf_knn.predict(X_test)
pred_dt  = clf_dt.predict(X_test)
pred_lr  = clf_lr.predict(X_test)
pred_svm = clf_svm.predict(X_test)
pred_nb  = clf_nb.predict(X_test)

# 성능
from sklearn.metrics import f1_score

score_knn = f1_score(y_test, pred_knn)
score_dt = f1_score(y_test, pred_knn)
score_lr = f1_score(y_test, pred_knn)
score_svm = f1_score(y_test, pred_knn)
score_nb = f1_score(y_test, pred_knn)

print(f'F1-Score - KNN: {score_knn:.3f}\
        \nF1-Score - DT : {score_dt:.3f}\
        \nF1-Score - LR : {score_lr:.3f}\
        \nF1-Score - SVM : {score_svm:.3f}\
        \nF1-Score - NB : {score_nb:.3f}')




F1-Score - KNN: 1.000        
F1-Score - DT : 1.000        
F1-Score - LR : 1.000        
F1-Score - SVM : 1.000        
F1-Score - NB : 1.000
